In [5]:
from ete3 import NCBITaxa, Tree, TreeStyle
ncbi =NCBITaxa()

def get_name(tax_id):
    ncbi =NCBITaxa()
    return (ncbi.translate_to_names([tax_id]))
get_name(1071380)        

[u'Tetrapisispora blattae CBS 6284']

In [6]:
import xml.dom.minidom as minidom
 
def getTitles(xml):   
    doc = minidom.parse(xml)
    node = doc.documentElement
    taxonomies = doc.getElementsByTagName("taxonomy")
 
    for taxonomy in taxonomies:
        sci_name = taxonomy.getElementsByTagName("scientific_name")[0]
        nodes = sci_name.childNodes
        for node in nodes:
            if node.nodeType == node.TEXT_NODE:
                print node.data
 
if __name__ == "__main__":
    document = 'NCBIPhylo.xml'
    getTitles(document)


5579
1043004
1043005


# Get node Stats

In [7]:
import pandas as pd

df=pd.read_csv("stats.csv", names=['taxid','CDS','CDS_Mean','exon','exon_Mean','gene','gene_Mean','mRNA','mRNA_Mean'])
taxid_list=df['taxid']

def get_leaves_taxid(nodeset):
    ncbi =NCBITaxa()
    df=pd.read_csv("stats.csv", names=['taxid','CDS','CDS_Mean','exon','exon_Mean','gene','gene_Mean','mRNA','mRNA_Mean'])
    taxid_list=df['taxid']
    gff_set=set()
    
    for nodeid in nodeset:
        for taxid in taxid_list:
            if nodeid in ncbi.get_lineage(taxid):
                gff_set.add(taxid)
    return gff_set

def get_node_stats(tax_id): 
    
    node_Name= ncbi.translate_to_names([int(tax_id)])
    info=df.loc[df['taxid'] == int(tax_id)].values.tolist()
    row=info[0]
    CDS=row[1]
    exon=row[3]
    gene=row[5]
    mRNA=row[7]
    tax2name= str(ncbi.translate_to_names([int(tax_id)]))
    return CDS,exon, gene, mRNA, tax2name

def get_node_stats2(tax_id):
    taxid_list=get_leaves_taxid(set([int(tax_id)]))
    avg_CDS= df[df['taxid'].isin(taxid_list)]['CDS'].mean()
    avg_exon= df[df['taxid'].isin(taxid_list)]['exon'].mean()
    avg_gene= df[df['taxid'].isin(taxid_list)]['gene'].mean()
    avg_mRNA= df[df['taxid'].isin(taxid_list)]['mRNA'].mean()
    tax2name= str(ncbi.translate_to_names([int(tax_id)]))
    
    return avg_CDS,avg_exon, avg_gene, avg_mRNA, tax2name


        
get_node_stats(1257118)
get_node_stats2(1257118)

(105766.0,
 105766.0,
 15655.0,
 14974.0,
 "[u'Acanthamoeba castellanii str. Neff']")

# Annotate clade with new property

In [8]:
import xml.dom.minidom as minidom
 
def getTitles(xml):   
    doc = minidom.parse(xml)
    node = doc.documentElement
    clades = doc.getElementsByTagName("clade")
    
    taxonomy_list=[]
    for clade in clades:
        taxid=""
        taxonomy = clade.getElementsByTagName("taxonomy")[0]
        taxonomy_list.append(taxonomy)
        sci_name = taxonomy.getElementsByTagName("scientific_name")[0]
        nodes = sci_name.childNodes
        for node in nodes:
            if node.nodeType == node.TEXT_NODE:
                taxid= node.data

        # <property ref="avg_CDS" datatype="xsd:string" applies_to="node">avg_CDS</property>        
        avg_CDS,avg_exon, avg_gene, avg_mRNA, tax2name= get_node_stats2(taxid)
 
        x = doc.createElement("name")  
        txt = doc.createTextNode(tax2name)  
        x.appendChild(txt)  
        clade.appendChild(x)  

        x = doc.createElement("property")  
        x.setAttribute("ref", "avg_CDS")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(avg_CDS))  
        x.appendChild(txt)  
        clade.appendChild(x)  
        
        x = doc.createElement("property")  
        x.setAttribute("ref", "avg_exon")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(avg_exon))  
        x.appendChild(txt)  
        clade.appendChild(x)
        
        x = doc.createElement("property")  
        x.setAttribute("ref", "avg_gene")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(avg_gene))  
        x.appendChild(txt)  
        clade.appendChild(x) 
    
        x = doc.createElement("property")  
        x.setAttribute("ref", "avg_mRNA")
        x.setAttribute("datatype", "xsd:string")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(str(avg_mRNA))  
        x.appendChild(txt)  
        clade.appendChild(x)  
        
        #<property ref="See also1" applies_to="clade" datatype="xsd:anyURI"> url </property>
        ncbi_url="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=" + str(taxid)
        x = doc.createElement("property")  
        x.setAttribute("ref", "see ncbi:")
        x.setAttribute("datatype", "xsd:anyURI")
        x.setAttribute("applies_to", "node")
        txt = doc.createTextNode(ncbi_url)  
        x.appendChild(txt)  
        clade.appendChild(x)
        
        
    print(doc.toxml())
    with open ("NCBIPhylo_minidom.xml", 'w') as f:
        doc.writexml(f)

if __name__ == "__main__":
    document = 'tree.xml'
    getTitles(document)

<?xml version="1.0" ?><phyloxml xmlns="http://www.phyloxml.org" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.phyloxml.org http://www.phyloxml.org/1.20/phyloxml.xsd">
<phylogeny rerootable="true" rooted="false">
  <clade>
    <clade>
      <branch_length>1.0</branch_length>
      <taxonomy>
        <scientific_name>33634</scientific_name>
      </taxonomy>
      <clade>
        <branch_length>1.0</branch_length>
        <taxonomy>
          <scientific_name>4762</scientific_name>
        </taxonomy>
        <clade>
          <branch_length>1.0</branch_length>
          <taxonomy>
            <scientific_name>4783</scientific_name>
          </taxonomy>
          <clade>
            <branch_length>1.0</branch_length>
            <taxonomy>
              <scientific_name>67593</scientific_name>
            </taxonomy>
          <name>[u'Phytophthora sojae']</name><property applies_to="node" datatype="xsd:string" ref="avg_CDS">62858.0</property><prop